# PV segmentation

Comparison for finloop - YOLO-segment vs. YOLOdetect + SAM

source (Huggingface): finloop/yolov8s-seg-solar-panels (aka Rzeszów model)

credits: https://blog.roboflow.com/how-to-use-yolov8-with-sam/ (Roboflow)

XI 25

*MD*

note: a dual-use model (detection + instance segmentation)

requires OBB-versioned datasets

## libs

In [ ]:
# %pip install numpy
# %pip install pandas
# %pip install ultralytics

In [ ]:
import matplotlib.pyplot as plt
import cv2
from ultralytics import YOLO, SAM
import torch
from os import listdir
from os.path import isfile, join

In [ ]:
dev = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
dev

device(type='cuda', index=0)

## defs

### data

In [ ]:
pilot = 'pilotPV_panels.v1i.yolov8-obb/test/images/*.jpg'
rzeszow_test = 'rzeszowSolar panels seg.v2i.yolov8-obb/test/images/*.jpg'
rzeszow_train = 'rzeszowSolar panels seg.v2i.yolov8-obb/train/images/*.jpg'
rzeszow_valid = 'rzeszowSolar panels seg.v2i.yolov8-obb/valid/images/*.jpg'
synth_test = 'auto_pv_to_fine_tunning.v4i.yolov8-obb/test/images/*.jpg'
synth_train = 'auto_pv_to_fine_tunning.v4i.yolov8-obb/train/images/*.jpg'
synth_valid = 'auto_pv_to_fine_tunning.v4i.yolov8-obb/valid/images/*.jpg'

In [ ]:
proste_1_zdj = "rzeszowSolar panels seg.v2i.yolov8-obb/test/images/022_jpg.rf.f76ab3a091f7c7931d2c26cc7375842b.jpg"
data_p = proste_1_zdj
pth = proste_1_zdj
nazwa = "proste_1_zdj"

### model

In [ ]:
model_pt = "best.pt"

In [ ]:
model = YOLO(model=model_pt, task="detect", verbose=True)

In [ ]:
sam = SAM("sam_b.pt")

### segment analysis

In [ ]:
def sum_pv_segments(pth, nazwa, model=model, disp_img=False, print_info=False):
    pv_area = 0
    image = cv2.cvtColor(cv2.imread(pth), cv2.COLOR_BGR2RGB)
    image = torch.tensor(image, device=dev)
    img_w, img_h, _ = image.shape
    img_w_cp, img_h_cp = img_w, img_h 
    results = model(pth, save=print_info, name=nazwa, stream=True, imgsz=(img_w, img_h))
    # YOLO requires dims of 32 multiply
    if img_h % 32 != 0 or img_w % 32 != 0:
        disp_img = False
    if img_h % 32 != 0:
        img_h = img_h // 32 + 1
        img_h *= 32
    if img_w % 32 != 0:
        img_w = img_w // 32 + 1
        img_w *= 32
    for i, res in enumerate(results):
        if print_info:
            print(i)
        if res is not None and res.masks is not None:
            for j, msk in enumerate(res.masks):
                binary_mask = torch.where(msk.data > 0.5, 1, 0)
                pv_area += binary_mask.sum().div(img_w_cp*img_h_cp) # percentage
                if print_info:
                    print('\t', j, binary_mask)
                if disp_img:
                    binary_mask = binary_mask.data
                    bcg_white = torch.ones_like(image)*255
                    new_image = bcg_white * (1 - binary_mask[..., torch.newaxis]) + image * binary_mask[..., torch.newaxis]
                    plt.imshow(new_image.reshape((img_w, img_h, 3)).cpu())
                    plt.title(f"Mask {j} in {pth[pth.rfind('/'):]}")
                    plt.axis('off')
                    plt.show()
        if print_info:
            print(pv_area.item())
        if disp_img:
            plt.imshow(image.cpu())
            plt.title(f"base img {pth[pth.rfind('/'):]}")
            plt.axis('off')
            plt.show()
    return pv_area

In [ ]:
def sum_pv_segments_sam(pth, nazwa, model=model, disp_img=False, print_info=False):
    pv_area = 0
    image = cv2.cvtColor(cv2.imread(pth), cv2.COLOR_BGR2RGB)
    image = torch.tensor(image, device=dev)
    img_w, img_h, _ = image.shape
    img_w_cp, img_h_cp = img_w, img_h 
    yolo_results = model(pth, save=print_info, name=nazwa, stream=True, imgsz=(img_w, img_h))
    # YOLO requires dims of 32 multiply
    if img_h % 32 != 0 or img_w % 32 != 0:
        disp_img = False
    if img_h % 32 != 0:
        img_h = img_h // 32 + 1
        img_h *= 32
    if img_w % 32 != 0:
        img_w = img_w // 32 + 1
        img_w *= 32
    for i, res in enumerate(yolo_results):
        for j, box in enumerate(res.boxes):
            sam_results = sam.predict(source=pth, bboxes=box.xyxy)
            if res is not None and sam_results is not None:
                for jj, sam_res in enumerate(sam_results):
                    binary_mask = torch.where(sam_res.masks.data == True, 1, 0)
                    pv_area += binary_mask.sum().div(img_w_cp*img_h_cp) # percentage
                    if print_info:
                        print('\t', j, binary_mask.shape)
                    if disp_img:
                        binary_mask = binary_mask.data
                        print(binary_mask.shape)
                        bcg_white = torch.ones_like(image)*255
                        new_image = bcg_white * (1 - binary_mask[..., torch.newaxis]) + image * binary_mask[..., torch.newaxis]
                        plt.imshow(new_image.reshape((img_w, img_h, 3)).cpu())
                        plt.title(f"Mask {jj} in {pth[pth.rfind('/'):]}")
                        plt.axis('off')
                        plt.show()
            if print_info:
                print(pv_area.item())
        if disp_img:
            plt.imshow(image.cpu())
            plt.title(f"base img {pth[pth.rfind('/'):]}")
            plt.axis('off')
            plt.show()
    return pv_area

In [ ]:
def sum_pv_segments_batch(imgs, nazwa="no_info_run", model=model, disp_img=False, print_info=False, display_coef=100):
    pv_area = 0
    if isinstance(imgs, str):
        if "*" in imgs:
            imgs = imgs.split("*")[0]
            imgs = [join(imgs, f) for f in listdir(imgs) if isfile(join(imgs, f)) and f.endswith('jpg')]
            for i, img in enumerate(imgs):
                do_disp = i % display_coef == 0
                pv_area += sum_pv_segments(img, nazwa, model, disp_img and do_disp, print_info)
            return pv_area
        return sum_pv_segments(imgs, nazwa, model, disp_img, print_info)
    else:
        for img in imgs:
            pv_area += sum_pv_segments(img, nazwa, model, disp_img, print_info)
    return pv_area

In [ ]:
def sum_pv_segments_batch_sam(imgs, nazwa="no_info_run", model=model, disp_img=False, print_info=False, display_coef=100):
    pv_area = 0
    if isinstance(imgs, str):
        if "*" in imgs:
            imgs = imgs.split("*")[0]
            imgs = [join(imgs, f) for f in listdir(imgs) if isfile(join(imgs, f)) and f.endswith('jpg')]
            for i, img in enumerate(imgs):
                do_disp = i % display_coef == 0
                pv_area += sum_pv_segments_sam(img, nazwa, model, disp_img and do_disp, print_info)
            return pv_area
        return sum_pv_segments_sam(imgs, nazwa, model, disp_img, print_info)
    else:
        for img in imgs:
            pv_area += sum_pv_segments_sam(img, nazwa, model, disp_img, print_info)
    return pv_area

## pilot

In [ ]:
sum_pv_segments_batch(pilot, "pilot_segment")

In [ ]:
sum_pv_segments_batch_sam(pilot, "pilot_segment")

## synthetic

### train

In [ ]:
sum_pv_segments_batch(synth_train)

In [ ]:
sum_pv_segments_batch_sam(synth_train)

### val

In [ ]:
sum_pv_segments_batch(synth_valid)

In [ ]:
sum_pv_segments_batch_sam(synth_valid)

### test

In [ ]:
sum_pv_segments_batch(synth_test)

In [ ]:
sum_pv_segments_batch_sam(synth_test)

## Rzeszów

### train

In [ ]:
# sum_pv_segments_batch(rzeszow_train)

In [ ]:
# sum_pv_segments_batch_sam(rzeszow_train)

### val

In [ ]:
sum_pv_segments_batch(rzeszow_valid)

In [ ]:
sum_pv_segments_batch_sam(rzeszow_valid)

### test

In [ ]:
sum_pv_segments_batch(rzeszow_test)

In [ ]:
sum_pv_segments_batch_sam(rzeszow_test)